Global Constants

In [1]:
import codecs, re, sys

sys.path.append("~/Documents/GitHub/Voices-Now-SRT-Generator/Scripts")

from ProTools.Timecode import Timecode, OffsetType

from Captions.CaptionMaker import CaptionMaker
from Captions.LanguageManager import LanguageManager

LANGUAGE_MANAGER = LanguageManager()


PROJECT_PATH = "/Volumes/VONGeneral/LDS/~Current Projects/MFD Training Videos for Coordinating Councils/Scripts/Translated Scripts/{0}/{1}/MFD_TrainingCouncil{2}.{3}"

def split_languages(timecode_offset: Timecode = None, timecode_offset_type: OffsetType = OffsetType.NONE, languages = []):
    for lang_code in languages:
        full_lang = LANGUAGE_MANAGER.get_lang(lang_code)
        max_line_len = LANGUAGE_MANAGER.get_max_line_count(lang_code)
        split = LANGUAGE_MANAGER.get_splitter(lang_code)

        combined_srt = PROJECT_PATH.format(full_lang, "Initial SRTs", "Combined", "srt")

        with codecs.open(combined_srt, "w+", encoding="utf-8") as combined_file:
            srt_offset = 0

            for i in range(1, 4):
                script_name = PROJECT_PATH.format(full_lang, "Scripts", "0" + str(i), "docx")
                edl_name = PROJECT_PATH.format("~Template", "EDLs", "0" + str(i), "txt")
                srt_name = PROJECT_PATH.format(full_lang, "Initial SRTs", "0" + str(i), "srt")                

                print(script_name)

                srt_maker = CaptionMaker(script_name, edl_name, "EDL", lang_code, srt_name, max_line_len, split)
                srt_offset = srt_maker.create_captions(timecode_offset, timecode_offset_type, srt_offset)

                with codecs.open(srt_name, "r", encoding="utf-8") as srt_file:
                    combined_file.write(srt_file.read())

def no_split_languages(timecode_offset: tuple = None, languages = []):
    for lang_code in languages:
        full_lang = LANGUAGE_MANAGER.get_lang(lang_code)

        combined_srt = PROJECT_PATH.format(full_lang, "Initial SRTs", "Combined", "srt")

        with codecs.open(combined_srt, "w+", encoding="utf-8") as combined_file:
            srt_offset = 0

            timecode_offsets = [None, ('dly', Timecode.from_total_frames("00:04:33;24")), ('dly', Timecode.from_total_frames("00:08:16;11"))]

            for i in range(1, 4):
                script_name = PROJECT_PATH.format(full_lang, "Scripts", "0" + str(i), "docx")
                # edl_name = project_path.format("~Template", "EDLs", "0" + str(i), "txt")
                srt_name = PROJECT_PATH.format(full_lang, "Initial SRTs", "0" + str(i), "srt")

                print(script_name)

                srt_maker = CaptionMaker(script_name, None, "MRK", lang_code, srt_name, 44, False)
                srt_offset = srt_maker.create_captions(timecode_offsets[i-1], srt_offset)

                with codecs.open(srt_name, "r", encoding="utf-8") as srt_file:
                    combined_file.write(srt_file.read())

def offset_single_srt(srt_name: str, offset: Timecode):
    pattern = r"\d{2}:\d{2}:\d{2},\d{3}"

    with codecs.open(srt_name, "r", encoding="utf-8") as srt_file:
        srt_data = srt_file.read()

    offset_srt_data = re.sub(pattern, lambda match: (Timecode.from_total_frames(match.group()) - offset).get_timecode_in_ms(), srt_data)

    with codecs.open(srt_name, "w", encoding="utf-8") as srt_file:
        srt_file.write(offset_srt_data)

def offset_srts(offset: Timecode, languages: list = []):
    for lang_code in languages:
        full_lang = LANGUAGE_MANAGER.get_lang(lang_code)

        for i in range(1, 4):
            srt_name = PROJECT_PATH.format(full_lang, "Initial SRTs", "0" + str(i), "srt")
            offset_single_srt(srt_name, offset)
        
        offset_single_srt(PROJECT_PATH.format(full_lang, "Initial SRTs", "Combined", "srt"), offset)
            
def specific_language(lang_code: str, timecode_offset: tuple = None):
    full_lang = LANGUAGE_MANAGER.get_lang(lang_code)

    combined_srt = PROJECT_PATH.format(full_lang, "Initial SRTs", "Combined", "srt")

    with codecs.open(combined_srt, "w+", encoding="utf-8") as combined_file:
        srt_offset = 0

        for i in range(1, 4):
            script_name = PROJECT_PATH.format(full_lang, "Scripts", "0" + str(i), "docx")
            edl_name = PROJECT_PATH.format(full_lang, "EDLs", "0" + str(i), "txt")
            srt_name = PROJECT_PATH.format(full_lang, "Initial SRTs", "0" + str(i), "srt")
            split = LANGUAGE_MANAGER.get_splitter(lang_code)

            print(script_name)

            srt_maker = CaptionMaker(script_name, edl_name, "EDL", lang_code, srt_name, 44, split)
            srt_offset = srt_maker.create_captions(timecode_offset, srt_offset)

            with codecs.open(srt_name, "r", encoding="utf-8") as srt_file:
                combined_file.write(srt_file.read())

/Users/studiod/Documents/GitHub/Voices-Now-SRT-Generator/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:


languages = ["FRA"] # ["CEB", "FRA", "DEU", "ITA", "KHM", "KOR", "MLG", "MAH", "POR", "RUS", "SMO", "SPA", "TGL", "THA", "TON", "UKR", "VIE"]
no_split = ["JPN", "CMN"]

weird_placement = Timecode.from_string("00:00:14;25")

# offsets = [Timecode.from_frames("00:00:00;00"),
#            Timecode.from_frames("00:04:33;24"),
#            Timecode.from_frames("00:08:16;11")]

# specific_language("RUS", ("adv", weird_placement))
split_languages(weird_placement, OffsetType.ADVANCE, languages)



DEBUG:root:EDL: 1		01:00:25;18
DEBUG:root:EDL: 2		01:00:31;21
DEBUG:root:EDL: 3		01:00:36;10
DEBUG:root:EDL: 4		01:00:41;21
DEBUG:root:EDL: 5		01:00:48;25
DEBUG:root:EDL: 6		01:00:54;12
DEBUG:root:EDL: 7		01:01:02;19
DEBUG:root:EDL: 8		01:01:08;25
DEBUG:root:EDL: 9		01:01:13;01
DEBUG:root:EDL: 10		01:01:20;28
DEBUG:root:EDL: 11		01:01:27;24
DEBUG:root:EDL: 12		01:01:38;07
DEBUG:root:EDL: 13		01:01:45;13
DEBUG:root:EDL: 14		01:02:01;08
DEBUG:root:EDL: 15		01:02:11;17
DEBUG:root:EDL: 16		01:02:19;17
DEBUG:root:EDL: 17		01:02:25;00
DEBUG:root:EDL: 18		01:02:34;03
DEBUG:root:EDL: 19		01:02:40;09
DEBUG:root:EDL: 20		01:02:45;21
DEBUG:root:EDL: 21		01:02:49;19
DEBUG:root:EDL: 22		01:03:01;20
DEBUG:root:EDL: 23		01:03:14;04
DEBUG:root:EDL: 24		01:03:20;00
DEBUG:root:EDL: 25		01:03:28;17
DEBUG:root:EDL: 26		01:03:36;28
DEBUG:root:EDL: 27		01:03:44;08
DEBUG:root:EDL: 28		01:03:55;12
DEBUG:root:EDL: 29		01:04:04;15
DEBUG:root:EDL: 30		01:04:11;16
DEBUG:root:EDL: 31		01:04:27;23


/Volumes/VONGeneral/LDS/~Current Projects/MFD Training Videos for Coordinating Councils/Scripts/Translated Scripts/French/Scripts/MFD_TrainingCouncil01.docx
/Volumes/VONGeneral/LDS/~Current Projects/MFD Training Videos for Coordinating Councils/Scripts/Translated Scripts/French/Scripts/MFD_TrainingCouncil02.docx


DEBUG:root:EDL: 1		01:04:43;22
DEBUG:root:EDL: 2		01:04:49;24
DEBUG:root:EDL: 3		01:04:56;00
DEBUG:root:EDL: 4		01:05:02;11
DEBUG:root:EDL: 5		01:05:15;07
DEBUG:root:EDL: 6		01:05:25;16
DEBUG:root:EDL: 7		01:05:31;18
DEBUG:root:EDL: 8		01:05:34;24
DEBUG:root:EDL: 9		01:05:44;17
DEBUG:root:EDL: 10		01:05:57;00
DEBUG:root:EDL: 11		01:06:04;29
DEBUG:root:EDL: 12		01:06:13;29
DEBUG:root:EDL: 13		01:06:25;23
DEBUG:root:EDL: 14		01:06:32;02
DEBUG:root:EDL: 15		01:06:39;12
DEBUG:root:EDL: 16		01:06:47;06
DEBUG:root:EDL: 17		01:06:54;27
DEBUG:root:EDL: 18		01:06:59;08
DEBUG:root:EDL: 19		01:07:08;15
DEBUG:root:EDL: 20		01:07:14;18
DEBUG:root:EDL: 21		01:07:18;08
DEBUG:root:EDL: 22		01:07:24;19
DEBUG:root:EDL: 23		01:07:28;21
DEBUG:root:EDL: 24		01:07:34;27
DEBUG:root:EDL: 25		01:07:44;22
DEBUG:root:EDL: 26		01:07:50;25
DEBUG:root:EDL: 27		01:07:55;09
DEBUG:root:EDL: 28		01:08:05;04
DEBUG:root:EDL: 1		01:08:25;10
DEBUG:root:EDL: 2		01:08:29;13
DEBUG:root:EDL: 3		01:08:37;13
DEBUG:root:EDL: 4		0

/Volumes/VONGeneral/LDS/~Current Projects/MFD Training Videos for Coordinating Councils/Scripts/Translated Scripts/French/Scripts/MFD_TrainingCouncil03.docx
